In [4]:
#set up environment
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal as sig
from scipy.io import loadmat, savemat
import warnings
#from Interictal_Spike_Analysis.HUMAN.working_feat_extract_code.functions.ied_fx_v3 import value_basis_multiroi
warnings.filterwarnings('ignore')
import seaborn as sns
#get all functions 
import sys, os
code_path = os.path.dirname('/mnt/leif/littlab/users/aguilac/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/functions/')
sys.path.append(code_path)
from ied_fx_v3 import *
pd.options.display.max_rows = 999

In [5]:
#Setup ptnames and directory
data_directory = ['/mnt/leif/littlab/users/aguilac/Projects/FC_toolbox/results/mat_output_v2', '/mnt/leif/littlab/data/Human_Data']
pt = pd.read_csv('/mnt/leif/littlab/users/aguilac/Projects/FC_toolbox/results/mat_output_v2/pt_data/pkl_list.csv') #pkl list is our list of the transferred data (mat73 -> pickle)
pt = pt['pt'].to_list()
blacklist = ['HUP101' ,'HUP112','HUP115','HUP124','HUP144','HUP147','HUP149','HUP155','HUP176','HUP193','HUP194','HUP195','HUP198','HUP208','HUP212','HUP216','HUP217','HUP064','HUP071','HUP072','HUP073','HUP085','HUP094']
ptnames = [i for i in pt if i not in blacklist] #use only the best EEG signals (>75% visually validated)

#establish ROI's
roiL_mesial = [' left entorhinal ', ' left parahippocampal ' , ' left hippocampus ', ' left amygdala ', ' left perirhinal ']
roiL_lateral = [' left inferior temporal ', ' left superior temporal ', ' left middle temporal ', ' left fusiform ', ' left transverse temporal '] #lingual??
roiR_mesial = [' right entorhinal ', ' right parahippocampal ', ' right hippocampus ', ' right amygdala ', ' right perirhinal ']
roiR_lateral = [' right inferior temporal ', ' right superior temporal ', ' right middle temporal ', ' right fusiform ', ' right transverse temporal ']
emptylabel = ['EmptyLabel','NaN']
L_OC = [' left inferior parietal ', ' left postcentral ', ' left superior parietal ', ' left precentral ', ' left rostral middle frontal ', ' left pars triangularis ', ' left supramarginal ', ' left insula ', ' left caudal middle frontal ', ' left posterior cingulate ', ' left lateral orbitofrontal ', ' left lateral occipital ', ' left cuneus ']
R_OC = [' right inferior parietal ', ' right postcentral ', ' right superior parietal ', ' right precentral ', ' right rostral middle frontal ', ' right pars triangularis ', ' right supramarginal ', ' right insula ', ' right caudal middle frontal ', ' right posterior cingulate ', ' right lateral orbitofrontal ', ' right lateral occipital ', ' right cuneus ']

roilist = [roiL_mesial, roiL_lateral, roiR_mesial, roiR_lateral, L_OC, R_OC, emptylabel]

In [8]:
ptnames

n = 15
lists_ptnames = (divide_chunks(ptnames, n))
(lists_ptnames)

[['HUP100',
  'HUP102',
  'HUP105',
  'HUP143',
  'HUP184',
  'HUP106',
  'HUP185',
  'HUP107',
  'HUP145',
  'HUP186',
  'HUP108',
  'HUP146',
  'HUP187',
  'HUP110',
  'HUP188'],
 ['HUP111',
  'HUP148',
  'HUP189',
  'HUP190',
  'HUP113',
  'HUP150',
  'HUP191',
  'HUP114',
  'HUP151',
  'HUP192',
  'HUP152',
  'HUP116',
  'HUP153',
  'HUP117',
  'HUP154'],
 ['HUP118',
  'HUP196',
  'HUP156',
  'HUP197',
  'HUP157',
  'HUP120',
  'HUP199',
  'HUP158',
  'HUP121',
  'HUP159',
  'HUP201',
  'HUP122',
  'HUP160',
  'HUP202',
  'HUP123'],
 ['HUP161',
  'HUP203',
  'HUP162',
  'HUP204',
  'HUP126',
  'HUP163',
  'HUP205',
  'HUP127',
  'HUP164',
  'HUP206',
  'HUP128',
  'HUP165',
  'HUP207',
  'HUP129',
  'HUP166'],
 ['HUP130',
  'HUP167',
  'HUP209',
  'HUP131',
  'HUP168',
  'HUP210',
  'HUP132',
  'HUP218',
  'HUP169',
  'HUP133',
  'HUP211',
  'HUP170',
  'HUP065',
  'HUP134',
  'HUP171'],
 ['HUP213',
  'HUP068',
  'HUP135',
  'HUP172',
  'HUP070',
  'HUP214',
  'HUP136',
  'HUP215',

In [4]:
def checker2(ptnames):
     
    data_directory = ['/mnt/leif/littlab/users/aguilac/Projects/FC_toolbox/results/mat_output_v2', '/mnt/leif/littlab/data/Human_Data']
    final_list = []
    for ptname in ptnames:
        print("starting: {}".format(ptname))
        #load data
        if ptname == 'HUP106':
            spike, brain_df, soz, ids  = load_ptall(ptname, data_directory, change_label = True)
        else:
            spike, brain_df, soz, ids = load_ptall(ptname, data_directory, change_label = False)
        
        if isinstance(brain_df, pd.DataFrame) == False:
            continue
        else:
            #get roi values
            vals, idxch, chs, select_oi = value_basis_multiroi(spike, brain_df, roilist)

            #get counts per region
            count_roi = []
            for roi in select_oi:
                count_roi.append(np.size(roi))

            final_list.append([ids, count_roi])

    return final_list

In [5]:
#final_list = checker2(ptnames)
#id_count = pd.DataFrame(final_list, columns = ['id', 'count'])
id_count_thres1 = pd.read_csv('/mnt/leif/littlab/users/aguilac/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/feasbility tests/all_pts_counts.csv', index_col = 0)
id_count_left = id_count_thres1[(id_count_thres1['lateralization'] == 'left') & ((id_count_thres1['R Mesial'] < 6) | (id_count_thres1['R Lateral'] < 6) | (id_count_thres1['R OC'] < 6))]
id_count_right = id_count_thres1[(id_count_thres1['lateralization'] == 'right') & ((id_count_thres1['L Mesial'] < 6) | (id_count_thres1['L Lateral'] < 6) | (id_count_thres1['L OC'] < 6))]

In [76]:
#id_count_clean = unnesting(id_count, ['count'], axis = 0)
#id_count_clean = unnesting(id_count_clean, ['id'], axis = 0)
#id_count_clean = id_count_clean.rename(columns = {'id0':'hup id', 'id1':'rid id', 'count0':'L Mesial', 'count1':'L Lateral', 'count2':'R Mesial', 'count3':'R Lateral', 'count4':'L OC', 'count5':'R OC', 'count6':'Empty Label'})
#id_count_thres1 = id_count_clean#[(id_count_clean['L Mesial'] < 6) | (id_count_clean['L Lateral'] < 6) | (id_count_clean['R Mesial'] < 6) | (id_count_clean['R Lateral'] < 6) | (id_count_clean['L OC'] < 6) | (id_count_clean['R OC'] < 6)]


In [77]:
#akash_ptlist = pd.read_csv('/mnt/leif/littlab/users/aguilac/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/feasbility tests/akash_bicov_master.csv', index_col = 0)
#id_count_thres1 = id_count_thres1.merge(akash_ptlist[['hup_id', 'region','lateralization']], left_on = 'hup id', right_on = 'hup_id', how = 'right', indicator = True)#.drop(columns = ['hup_id', '_merge'])

In [72]:
#id_count_thres1.to_csv('/mnt/leif/littlab/users/aguilac/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/feasbility tests/all_pts_counts.csv')
#d_count_thres1 = pd.read_csv('/mnt/leif/littlab/users/aguilac/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/feasbility tests/all_pts_counts.csv', index_col = 0)

In [6]:
id_count_thres1

,hup id,rid id,L Mesial,L Lateral,R Mesial,R Lateral,L OC,R OC,Empty Label,region,lateralization
0,HUP100,sub-RID0106,152,182,104,8,151,5,201,diffuse,left
1,HUP105,sub-RID0101,199,61,366,62,7,2,247,temporal neocortical,right
2,HUP143,sub-RID0206,274,106,64,191,8,21,319,multifocal,bilateral
3,HUP184,sub-RID0508,796,39,0,0,12,0,152,temporal,left
4,HUP106,sub-RID0143,425,266,62,0,19,0,223,temporal neocortical,left
5,HUP185,sub-RID0332,248,245,124,71,22,0,283,mesial temporal,left
6,HUP107,sub-RID0117,35,1,337,288,0,30,293,temporal neocortical,right
7,HUP145,sub-RID0167,0,0,1,0,0,296,507,other cortex,right
8,HUP186,sub-RID0420,378,173,0,10,6,0,432,temporal,left
9,HUP108,sub-RID0165,467,13,248,21,5,1,241,mesial temporal,bilateral


In [69]:
print(id_count_thres1.count()['hup id'])
print(id_count_left.count()['hup id'])
print(id_count_right.count()['hup id'])

77
34
21


In [8]:
id_count_left

,hup id,rid id,L Mesial,L Lateral,R Mesial,R Lateral,L OC,R OC,Empty Label,region,lateralization
0,HUP100,sub-RID0106,152,182,104,8,151,5,201,diffuse,left
3,HUP184,sub-RID0508,796,39,0,0,12,0,152,temporal,left
4,HUP106,sub-RID0143,425,266,62,0,19,0,223,temporal neocortical,left
5,HUP185,sub-RID0332,248,245,124,71,22,0,283,mesial temporal,left
8,HUP186,sub-RID0420,378,173,0,10,6,0,432,temporal,left
12,HUP110,sub-RID0160,0,414,0,0,192,0,207,diffuse,left
13,HUP188,sub-RID0472,1,51,0,0,265,0,424,other cortex,left
16,HUP189,sub-RID0520,0,637,0,0,135,0,224,temporal neocortical,left
17,HUP190,sub-RID0405,694,73,24,1,26,3,174,mesial temporal,left
18,HUP113,sub-RID0190,227,540,0,0,93,0,115,mesial temporal,left


## appears that there is about 55 patients who have double coverage, yet they don't 

# Analysis for electrode level SOZ

In [4]:
brain_concat_df = pd.read_csv('/mnt/leif/littlab/users/aguilac/Projects/FC_toolbox/results/mat_output_v2/pt_data/master_elecs.csv')#, index_col = 0)

In [19]:
rid1_df = brain_concat_df[(brain_concat_df['rid'] == 'sub-RID0013') & (brain_concat_df['soz'] == True)]
rid1_df

,rid,name,mni_x,mni_y,mni_z,mm_x,mm_y,mm_z,vox_x,vox_y,vox_z,label,soz,resected,spike_rate,engel
4,sub-RID0013,MST01,4.300,-67.361,16.011,15.2559,-41.60630,-33.6340,131.741,64.3756,70.4205,right lingual,True,False,3.439490,1.0
5,sub-RID0013,MST02,11.385,-64.370,15.294,24.3965,-38.01080,-35.1712,141.101,68.0574,68.8833,right cerebellum exterior,True,False,3.985441,1.0
6,sub-RID0013,MST03,18.472,-61.429,15.077,33.5483,-34.57790,-35.7218,150.472,71.5727,68.3327,EmptyLabel,True,False,5.787079,1.0
69,sub-RID0013,RG50,36.512,-50.346,15.227,56.8630,-21.97300,-35.9334,174.347,84.4802,68.1211,EmptyLabel,True,False,0.436761,1.0
70,sub-RID0013,RG51,41.038,-51.202,19.140,62.7742,-23.69310,-28.1749,180.400,82.7187,75.8796,right inferior temporal,True,True,1.255687,1.0
81,sub-RID0013,RG62,43.015,-41.037,35.007,65.6348,-15.68830,2.5055,183.329,90.9156,106.5600,EmptyLabel,True,False,0.436761,1.0
82,sub-RID0013,RG63,41.740,-39.965,39.700,64.0762,-15.50760,11.6748,181.733,91.1007,115.7290,right supramarginal,True,False,1.091902,1.0
84,sub-RID0013,RH01,13.641,-38.475,22.379,27.4942,-10.58280,-22.5765,144.273,96.1437,81.4779,EmptyLabel,True,False,4.149227,1.0
85,sub-RID0013,RH02,17.187,-37.457,22.977,32.0854,-9.52078,-21.4396,148.974,97.2312,82.6149,EmptyLabel,True,False,11.246588,1.0
86,sub-RID0013,RH03,20.730,-36.713,22.530,36.6529,-8.54399,-22.3435,153.651,98.2314,81.7110,EmptyLabel,True,True,11.028207,1.0


In [26]:
#use load_ptall from ied_fx_v3 to load all patients into a dictionary
spike, brain_df, onsetzone, ids = load_ptall('HUP105', data_directory, change_label = False) #only change for 1 patient
print(ids)

['HUP105', 'sub-RID0101']


In [24]:
rid1_df = brain_concat_df[(brain_concat_df['rid'] == ids[-1]) & (brain_concat_df['soz'] == True)]
print

In [25]:
rid1_df

,rid,name,mni_x,mni_y,mni_z,mm_x,mm_y,mm_z,vox_x,vox_y,vox_z,label,soz,resected,spike_rate,engel
2327,sub-RID0101,RDNET01,23.070,-5.503,-50.431,20.5157,32.3067,-44.9967,116.877,143.066,33.8599,right inferior temporal,True,False,3.000000,NaN
2328,sub-RID0101,RDNET02,28.166,-6.573,-51.198,25.3395,31.4179,-46.0486,122.023,142.118,32.8081,right inferior temporal,True,False,3.573248,NaN
2329,sub-RID0101,RDNET03,33.181,-8.170,-50.768,30.1115,29.7552,-46.2742,127.113,140.345,32.5824,right inferior temporal,True,True,5.560510,NaN
2330,sub-RID0101,RDNET04,38.202,-8.666,-50.338,34.8979,29.1221,-46.2227,132.218,139.670,32.6339,right inferior temporal,True,True,5.751592,NaN
